<a href="https://colab.research.google.com/github/Melvinchen0404/Chess/blob/main/andomel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-chess ipywidgets datasets
!apt-get install stockfish

from google.colab import drive
drive.mount('/content/drive') #Download Syzygy 3-4-5 tablebase from the Internet Archive: https://archive.org/details/Syzygy345
#Upload to Google Drive
#Mount Google Drive to access the uploaded Syzygy 3-4-5 tablebase (Google Drive path is contained in endgame.py file and can be modified where necessary)

import chess
import endgame
from endgame import tablebase_evaluation, distance_to_zero

# Step 5: Test Syzygy 3-4-5 tablebase with a King vs King endgame
board = chess.Board("8/8/8/8/8/8/2K5/2k5 w - - 0 1")

eval_score = tablebase_evaluation(board)
print("Evaluation from tablebase:", eval_score)

dtz_score = distance_to_zero(board)
print("Depth to Zeroing (DTZ):", dtz_score) # Expected output: Evaluation from tablebase = 0 (King vs King is a draw); DTZ = 0 (there are no more moves remaining until the game reaches a draw or mate)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Evaluation from tablebase: 0
Depth to Zeroing (DTZ): 0


In [31]:
import sys
import chess
import argparse
import chess.svg
from IPython.display import display, SVG, clear_output
import ipywidgets as widgets
from datasets import load_dataset
import chess.engine  # Import Stockfish engine
from eval import best_move  # Import best_move from eval.py

# Load the chess openings dataset
dset = load_dataset("Lichess/chess-openings")
train_data = dset['train']

# Extract relevant data (name -> uci) from the dataset
openings_dict = {entry["name"]: entry["uci"] for entry in train_data}

# Create a text box for input
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')

# Create a button to submit the move or command
submit_button = widgets.Button(description="Submit")

# Create a VBox to display the chessboard and the input box
output_area = widgets.Output()

# Store the widgets in a container for persistent layout
layout_container = widgets.VBox([input_box, submit_button, output_area])

# Function to get user moves in UCI format
def get_user_moves(board):
    """Get user moves in UCI format."""
    return " ".join([move.uci() for move in board.move_stack])  # Get UCI moves

# Function to check if the moves match any known openings
def check_openings(user_moves):
    """Check if the given move sequence matches any known openings."""
    matched_openings = []
    for opening_name, opening_moves in openings_dict.items():
        if user_moves.startswith(opening_moves):  # Match the sequence
            matched_openings.append((opening_name, opening_moves))  # Append opening name and its moves
    return matched_openings  # Return matched openings

# Function to get the best move from Stockfish
def get_stockfish_best_move(board, depth=3):
    """Use Stockfish to get the best move."""
    with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as engine:
        result = engine.play(board, chess.engine.Limit(time=2.0))  # Adjust the time as necessary
        return result.move.uci()  # Return the best move in UCI format

# Function to update the chessboard and display best moves (Stockfish and eval.py)
output_lines = []  # Store the most recent output lines

def render_board_and_best_move(board, best_move_stockfish=None, best_move_eval=None, scale=0.6):
    """Render the chessboard as SVG with scaling and display best moves from Stockfish and eval.py."""
    global output_lines  # Use the global list to store lines

    # Prepare the message for Stockfish's best move
    if best_move_stockfish:
        stockfish_line = f"Best move from Stockfish engine: {best_move_stockfish}"
    else:
        stockfish_line = "No best move available from Stockfish engine."

    # Prepare the message for best move from eval.py
    if best_move_eval:
        eval_line = f"Best move from Andomel engine: {best_move_eval}"
    else:
        eval_line = "No best move available from Andomel engine."

    # Update the output lines with the new recommendations
    output_lines = [stockfish_line, eval_line]

    # Clear and update only the output_area content (keeping input and button intact)
    with output_area:
        clear_output(wait=True)  # Clear only the output area
        for line in output_lines:
            print(line)  # Display the retained lines
        display(SVG(chess.svg.board(board, size=400 * scale)))  # Display the chessboard

# Function to handle moves, single or sequence
def process_moves(msg, board):
    """
    Process single or sequence of moves.
    If the input is a single move, treat it as a single move.
    If the input contains space-separated moves, treat it as a sequence of moves.
    """
    msg = msg.strip()

    # Check for sequence of moves (space-separated moves)
    if ' ' in msg:  # If there are space-separated moves, it's a sequence
        moves = msg.split()
        for move_uci in moves:
            try:
                move = chess.Move.from_uci(move_uci)
                if move in board.legal_moves:
                    board.push(move)  # Apply the move
                    print(f"Move {move_uci} played.")
                    # Update the board and the best move display after each move
                    best_move_stockfish = get_stockfish_best_move(board)  # Get the best move from Stockfish
                    best_move_eval = best_move(board)  # Get the best move from eval.py
                    render_board_and_best_move(board, best_move_stockfish, best_move_eval)  # Update the display
                else:
                    print(f"Illegal move: {move_uci}. Try again.")
            except Exception as e:
                print(f"Invalid move format: {move_uci}. Error: {e}")
    else:  # Single move input (e.g., 'e2e4')
        try:
            move = chess.Move.from_uci(msg)
            if move in board.legal_moves:
                board.push(move)  # Apply the move
                print(f"Move {msg} played.")
                # Update the board and the best move display
                best_move_stockfish = get_stockfish_best_move(board)  # Get the best move from Stockfish
                best_move_eval = best_move(board)  # Get the best move from eval.py
                render_board_and_best_move(board, best_move_stockfish, best_move_eval)  # Update the display
            else:
                print(f"Illegal move: {msg}. Try again.")
        except Exception as e:
            print(f"Invalid move format: {msg}. Error: {e}")

# Define the command function
def command(depth: int, board: chess.Board, msg: str):
    """
    Accept UCI commands and respond.
    The board state is also updated.
    """
    msg = msg.strip()

    # Handle quit command
    if msg == "quit":
        sys.exit()  # Exit the program

    # Handle reset command
    if msg == "reset":
        board.reset()  # Reset the board to the starting position
        print("Game reset.")
        return

    # Handle undo command
    if msg == "undo":
        if len(board.move_stack) > 0:  # Check if there is at least one move to undo
            board.pop()  # Undo the last move
            print("Last move undone.")
        else:
            print("No moves to undo.")
        return

    # Handle UCI move inputs (like 'e2e4')
    if len(msg) >= 4 and len(msg) <= 5:  # UCI move (4 or 5 characters long)
        try:
            move = chess.Move.from_uci(msg)
            # Check if the move is legal before pushing it
            if move in board.legal_moves:
                board.push(move)  # Apply the move
                print(f"Move {msg} played.")
                # Update the board and the best move display
                best_move_stockfish = get_stockfish_best_move(board)  # Get the best move from Stockfish
                best_move_eval = best_move(board)  # Get the best move from eval.py
                render_board_and_best_move(board, best_move_stockfish, best_move_eval)  # Update the display
            else:
                print(f"Illegal move: {msg}. Try again.")
        except Exception as e:
            print(f"Invalid move format: {msg}. Error: {e}")

    # Handle "sequence" command (multiple moves)
    if msg == "sequence":
        print("Enter your move sequence (e.g., 'e2e4 e7e6 b1a3 ...'):")
        return  # Wait for the next input

    # After processing the move, show the board and both best moves
    best_move_stockfish = get_stockfish_best_move(board)  # Get the best move from Stockfish
    best_move_eval = best_move(board)  # Get the best move from eval.py

    # Update the display with both best moves
    render_board_and_best_move(board, best_move_stockfish, best_move_eval)

    # Handle openings
    user_moves = get_user_moves(board)  # Get user moves in UCI format
    matched_openings = check_openings(user_moves)  # Check if the move sequence matches any known openings

    # Print moves and check for openings
    print("Current moves in UCI:", user_moves)
    if matched_openings:
        for opening_name, uci in matched_openings:
            print(f"Match found! The opening is: {opening_name} (UCI: {uci})")
    else:
        print("No matching opening found.")

# Define the talk function to handle the input/output loop
def talk():
    """
    The main input/output loop.
    This implements a slice of the UCI protocol.
    """
    board = chess.Board()
    depth = get_depth()

    # Display initial board in the output_area
    render_board_and_best_move(board)

    def on_button_click(b):
        msg = input_box.value
        command(depth, board, msg)  # Process the command
        input_box.value = ""  # Clear the input box after submission

    submit_button.on_click(on_button_click)

    # Display the input box, button, and output area (board + updates)
    display(layout_container)

def get_depth():
    parser = argparse.ArgumentParser()
    parser.add_argument("--depth", default=3, type=int, help="provide an integer (default: 3)")
    args, unknown = parser.parse_known_args([arg for arg in sys.argv if arg.startswith("--")])
    return max(1, args.depth)


In [32]:
# Start the interactive UCI session
talk()

Enter your move sequence (e.g., 'e2e4 e7e6 b1a3 ...'):
Current moves in UCI: 
No matching opening found.
